In [8]:
import json
from pprint import pprint
import csv
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from xml.dom.minidom import parse, parseString
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

In [2]:
#SemEval
data_folder_sem_eval = 'data/sem_eval/'


#Map to ANGER, JOY, SADNESS, SUPRISE
mapping = {
'fairy_tales' : {'2' : 'ANGER', '3': 'NONE', '4': 'JOY', '5': 'NONE', '6': 'SADNESS', '7': 'SURPRISE'},
'ISEAR'       : {'anger': 'ANGER', 'disgust': 'NONE', 'fear' : 'NONE', 'joy': 'JOY', 'sadness': 'SADNESS', 'shame': 'NONE', 'guilt' : 'NONE'},
'sem_eval'    : {'anger' : 'ANGER', 'disgust': 'NONE', 'fear': 'NONE', 'joy': 'JOY', 'sadness': 'SADNESS', 'surprise': 'SURPRISE'},
'facebook'    : {'ANGRY' : 'ANGER', 'SAD' : 'SADNESS', 'HAHA' : 'JOY', 'WOW': 'SURPRISE', 'LOVE': 'JOY', 'LIKE' : 'NONE', 'NONE': 'NONE'}
}



In [3]:
reaction_types = ['NONE', 'LIKE', 'LOVE', 'WOW', 'HAHA', 'SAD', 'ANGRY', 'THANKFUL']
data = []
files = ['FoxNews', 'cnn', 'espn', 'nytimes']
for f in files:
    with open('data/facebook_data/{}.json'.format(f)) as data_file:    
        data.extend(json.load(data_file))

In [22]:
csv_data = []
categories = []
text = defaultdict(str)
result = defaultdict(list)
for post in data:
    csv_row = [post[0]['created_time'], post[0]['message']]
    for i, rt in enumerate(reaction_types):
        csv_row.append(post[0]['reactions'][i] / post[0]['reactions'][0])
    csv_row.append(reaction_types[post[0]['reactions'].index(max(post[0]['reactions'][2:]))])
    csv_data.append(csv_row)
    categories.append(reaction_types[post[0]['reactions'].index(max(post[0]['reactions'][2:]))])
    text[reaction_types[post[0]['reactions'].index(max(post[0]['reactions'][2:]))]] += post[0]['message']
    result[mapping['facebook'][reaction_types[post[0]['reactions'].index(max(post[0]['reactions'][2:]))]]].append((mapping['facebook'][reaction_types[post[0]['reactions'].index(max(post[0]['reactions'][2:]))]], post[0]['message'],max(post[0]['reactions'][2:]) / post[0]['reactions'][0]))

least_frequent = min([len(result[k])for k in result])
sorted_result = []
for k in result:
    sorted_result.extend(sorted(result[k], key = lambda x: int(x[2]))[:least_frequent])



In [23]:
'''Reading sem_eval'''
test_gold_doc = parse('data/sem_eval/AffectiveText.test/affectivetext_test.xml')
test_sentences = 'data/sem_eval/AffectiveText.test/affectivetext_test.emotions.gold'
train_gold_doc = parse('data/sem_eval/AffectiveText.trial/affectivetext_trial.xml')
train_sentences = 'data/sem_eval/AffectiveText.trial/affectivetext_trial.emotions.gold'
emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'sentence']

result_sem_eval = defaultdict(list)
with open(train_sentences) as f2:
    for line in f2.readlines():
        values = line.strip().split()
        result_sem_eval[values[0]] = [emotions[values[1:].index(max(values[1:]))]]

sentences = train_gold_doc.getElementsByTagName("instance")
for sentence in sentences:
    sid = sentence.getAttribute("id")
    result_sem_eval[sid].append(sentence.firstChild.nodeValue)


result_sem_eval = [(mapping['sem_eval'][emotion], sentence) for emotion, sentence in result_sem_eval.values()]

In [24]:
# define a pipeline combining a text feature extractor with a simple
# classifier

X = [x for y, x, s in sorted_result if y != 'NONE']
y = [y for y, x, s in sorted_result if y != 'NONE']

pipeline = Pipeline([
    ('features', FeatureUnion([
                ('words', TfidfVectorizer(stop_words = 'english'))
    ])),
    ('classifier', svm.SVC())
    ])


pipeline.fit(X, y)
X_sem = [x for y, x in result_sem_eval if y != 'NONE']
y_sem = [y for y, x in result_sem_eval if y != 'NONE']
y_pred = pipeline.predict(X_sem)
print(classification_report(y_sem, y_pred))
print(confusion_matrix(y_sem, y_pred))
accuracy_score(y_sem, y_pred)


             precision    recall  f1-score   support

      ANGER       0.22      0.51      0.31        41
        JOY       0.61      0.15      0.25        71
    SADNESS       0.41      0.32      0.36        44
   SURPRISE       0.27      0.34      0.30        35

avg / total       0.42      0.30      0.30       191

[[21  1 11  8]
 [36 11  5 19]
 [21  4 14  5]
 [17  2  4 12]]


0.30366492146596857